In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount= True)

Mounted at /content/drive


In [ ]:
# !pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth= GoogleAuth()
gauth.credentials= GoogleCredentials.get_application_default()
drive= GoogleDrive(gauth)

In [ ]:
!ls

adc.json  sample_data			  train_annotations_bbox_data.csv
drive	  test_annotations_bbox_data.csv  train_annotation.txt


In [ ]:
train_annotation_csv= drive.CreateFile({'id': '1NR5I_uFNQ-k1Q_D_9TkekLqZwH9iYbXF'})
train_annotation_csv.GetContentFile('train_annotations_bbox_data.csv')
test_annotation_csv= drive.CreateFile({'id': '1e5BGr66parAJTv9VqmK9TnvOs1rbzvgX'})
test_annotation_csv.GetContentFile('test_annotations_bbox_data.csv')

In [ ]:
import sys
import time
import numpy as np
import xml.etree.ElementTree as ET
from glob import glob
from optparse import OptionParser
import pickle
import math
import cv2
import copy
from matplotlib import pyplot as plt
import tensorflow as tf
import pandas as pd
import os

In [ ]:
# kk= pd.read_csv('train_annotations_bbox_data.csv')
# kk.head()

,filename,objects_name,xmin,xmax,ymin,ymax,img_width,img_height,img_depth,objects_label
0,000005.jpg,chair,263,324,211,339,500,375,3,1
1,000005.jpg,chair,165,253,264,372,500,375,3,1
2,000005.jpg,chair,5,67,244,374,500,375,3,1
3,000005.jpg,chair,241,295,194,299,500,375,3,1
4,000005.jpg,chair,277,312,186,220,500,375,3,1


In [ ]:
# def plot_bbox(img_id):
#   img_url = images_boxable.loc[images_boxable["ImageID"]==img_id]['OriginalURL'].values[0]
#   img = io.imread(img_url)
#   height, width, channel = img.shape
#   print(f"Image: {img.shape}")
#   bboxs = annotations_bbox[annotations_bbox['ImageID']==img_id]
#   for index, row in bboxs.iterrows():
#       xmin = row['XMin']
#       xmax = row['XMax']
#       ymin = row['YMin']
#       ymax = row['YMax']
#       xmin = int(xmin*width)
#       xmax = int(xmax*width)
#       ymin = int(ymin*height)
#       ymax = int(ymax*height)
#       label_name = row['LabelName']
#       class_series = class_descriptions[class_descriptions[0]==label_name]
#       class_name = class_series[1].values[0]
#       print(f"Coordinates: {xmin,ymin}, {xmax,ymax}")
#       cv2.rectangle(img, (xmin,ymin), (xmax,ymax), (255,0,0), 5)
#       font = cv2.FONT_HERSHEY_SIMPLEX
#       cv2.putText(img, class_name, (xmin,ymin-10), font, 3, (0,255,0), 5)
#   plt.figure(figsize=(15,10))
#   plt.title('Image with Bounding Box')
#   plt.imshow(img)
#   plt.axis("off")
#   plt.show()



In [ ]:
# ## This will be added in data-preprocessing

# def annotations_to_df(path, mode= 'train'):
    
#     annotations =glob(path) #'D:/Python Codes/DSP-I Projects/Faster_RCNN/DataSet/Annotations/*.xml') #glob('D:/Python Codes/DSP-I Projects/Mini Dataset/Annotations/*.xml')

#     df = [] ## data frame for all the objects in the picture and anchor boxes.
#     cnt = 1
#     for file in annotations:
#         #filename = file.split('/')[-1].split('.')[0] + '.jpg'
#         #filename = str(cnt) + '.jpg'
#         filename = file.split('\\')[-1]
#         filename =filename.split('.')[0] + '.jpg'
#         row = []
#         parsedXML = ET.parse(file)
#         img_width= int(parsedXML.getroot().find('size/width').text)
#         img_height= int(parsedXML.getroot().find('size/height').text)
#         img_depth= int(parsedXML.getroot().find('size/depth').text)
        
#         if img_depth == 3:
#             for node in parsedXML.getroot().iter('object'):
#                 objects_name = node.find('name').text
#                 xmin = int(node.find('bndbox/xmin').text) #/ img_width ## for naormalization (xmin,xmax)/img_width and (ymin,ymax)/img_height
#                 xmax = int(node.find('bndbox/xmax').text) #/ img_width
#                 ymin = int(node.find('bndbox/ymin').text) #/ img_height
#                 ymax = int(node.find('bndbox/ymax').text) #/ img_height
            

#                 row = [filename, objects_name, xmin, xmax, ymin, ymax, img_width, img_height, img_depth]
#                 df.append(row)
#                 cnt += 1
#         else:
#             continue
            

#     data = pd.DataFrame(df, columns=['filename', 'objects_name', 'xmin', 'xmax', 'ymin', 'ymax', 'img_width', 'img_height', 'img_depth'])
#     ## adding label column
#     data['objects_label']=data['objects_name']
#     lab=1
#     for name in data['objects_name'].unique():
        
#         data['objects_label'].replace({name: lab}, inplace=True)
#         lab+=1
    
#     if mode== 'train':
#         data.to_csv('train_annotations_bbox_data.csv', index=False)
#     elif mode== 'test':
#         data.to_csv('test_annotations_bbox_data.csv', index=False)
        
        
        

    
#     #print(data.head(50))
#     #data[['filename', 'objects_name', 'xmin', 'xmax', 'ymin', 'ymax', 'img_width', 'img_height', 'img_depth', 'objects_label']].to_csv('Annoted_data.csv', index=False)
        
#     return data

In [ ]:
#path= 'D:/Python Codes/DSP-I Projects/Faster_RCNN_1/DataSet/Annotations/*.xml'
train_annotation_path= '/content/drive/MyDrive/Faster_RCNN_try3/Train_DataSet/Annotations/*.xml' #'D:/Python Codes/DSP-I Projects/Faster_RCNN_try3/Train_DataSet/Annotations/*.xml'
test_annotation_path= '/content/drive/MyDrive/Faster_RCNN_try3/Test_DataSet/Annotations/*.xml' #'D:/Python Codes/DSP-I Projects/Faster_RCNN_try3/Test_DataSet/Annotations/*.xml'
path= train_annotation_path
# path= test_annotation_path
# inp_data= annotations_to_df(path, mode= 'train')
# inp_data= annotations_to_df(path, mode= 'test')

In [ ]:
inp_data= pd.read_csv('train_annotations_bbox_data.csv')
# inp_data= pd.read_csv('test_annotations_bbox_data.csv')

In [ ]:
#Creating full path for each image ID
#base_path='D:/Python Codes/DSP-I Projects/NNN/Mini Dataset/Images'
def creatingPath(x):
    #return ("D:/Python Codes/DSP-I Projects/Faster_RCNN_try3/Test_DataSet/Images/"+x)
    return ('/content/drive/MyDrive/Faster_RCNN_try3/Train_DataSet/Images/'+x)
    # return ('/content/drive/MyDrive/Faster_RCNN_try3/Test_DataSet/Images/'+x)

inp_data["path"] = inp_data["filename"].apply(creatingPath)

In [ ]:
# def creatingPath(x):
#     return ("'D:/Python Codes/DSP-I Projects/Faster_RCNN_for_Open_Images_Dataset_Keras-master/Test_DataSet/Images/"+x)

# inp_data["path"] = inp_data["filename"].apply(creatingPath)

In [ ]:
inp_data_dict= inp_data[['path', 'objects_name', 'img_width', 'img_height','xmin', 'xmax', 'ymin', 'ymax']]

In [ ]:
print(inp_data_dict)

                                                    path  ... ymax
0      /content/drive/MyDrive/Faster_RCNN_try3/Train_...  ...  339
1      /content/drive/MyDrive/Faster_RCNN_try3/Train_...  ...  372
2      /content/drive/MyDrive/Faster_RCNN_try3/Train_...  ...  374
3      /content/drive/MyDrive/Faster_RCNN_try3/Train_...  ...  299
4      /content/drive/MyDrive/Faster_RCNN_try3/Train_...  ...  220
...                                                  ...  ...  ...
15657  /content/drive/MyDrive/Faster_RCNN_try3/Train_...  ...  217
15658  /content/drive/MyDrive/Faster_RCNN_try3/Train_...  ...  124
15659  /content/drive/MyDrive/Faster_RCNN_try3/Train_...  ...  316
15660  /content/drive/MyDrive/Faster_RCNN_try3/Train_...  ...  187
15661  /content/drive/MyDrive/Faster_RCNN_try3/Train_...  ...  345

[15662 rows x 8 columns]


In [ ]:
def plot_bbox(img_id):
    img_url = img_id #images_boxable.loc[images_boxable["ImageID"]==img_id]['OriginalURL'].values[0]
    img= cv2.imread(img_url)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    height, width, channel = img.shape
    print(f"Image: {img.shape}")
    bboxs = inp_data_dict[inp_data_dict['path']== img_id]
    for index, row in bboxs.iterrows():
        xmin = row['xmin']
        xmax = row['xmax']
        ymin = row['ymin']
        ymax = row['ymax']
#         xmin = int(xmin*width)
#         xmax = int(xmax*width)
#         ymin = int(ymin*height)
#         ymax = int(ymax*height)
        label_name = row['objects_name']
#         class_series = class_descriptions[class_descriptions[0]==label_name]
        class_name = label_name #class_series[1].values[0]
        print(f"Coordinates: {xmin,ymin}, {xmax,ymax}")
        cv2.rectangle(img, (xmin,ymin), (xmax,ymax), (255,0,0), 2)
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(img, class_name, (xmin,ymin-10), font, 0.5, (0,255,0), 1)
    plt.figure(figsize=(15,10))
    plt.title('Image with Bounding Box')
    plt.imshow(img)
    plt.axis("off")
    plt.show() 

In [ ]:

train_df = inp_data_dict #[:int(inp_data_dict.shape[0]*0.9)]

#for training
with open("/content/drive/MyDrive/Faster_RCNN_try3/train_annotation.txt", "w+") as f: 
    for idx, row in train_df.iterrows():
        #img = cv2.imread(row['path']) 
        
        x1 = row['xmin'] #int(row['xmin']) 
        x2 = row['xmax'] #int(row['xmax']) 
        y1 = row['ymin'] #int(row['ymin']) 
        y2 = row['ymax'] #int(row['ymax'])
#         height = y2 - y1
#         width = x2 - x1

  #google_colab_file_path = 'drive/My Drive/AI/Dataset/Open Images Dataset v4 (Bounding Boxes)/train'
        fileName = row['path']
        className = row['objects_name']
        f.write(fileName + ',' + str(x1) + ',' + str(y1) + ',' + str(x2) + ',' + str(y2) + ',' + className + '\n')

In [ ]:
# test_df = inp_data_dict  #inp_data_dict[:int(inp_data_dict.shape[0]*0.9)]

# #for training
# with open("/content/drive/MyDrive/Faster_RCNN_try3/test_annotation.txt", "w+") as f: 
#     for idx, row in test_df.iterrows():
#         #img = cv2.imread(row['path']) 
        
#         x1 = row['xmin'] #int(row['xmin']) 
#         x2 = row['xmax'] #int(row['xmax']) 
#         y1 = row['ymin'] #int(row['ymin']) 
#         y2 = row['ymax'] #int(row['ymax'])
#         height = y2 - y1
#         width = x2 - x1

#   #google_colab_file_path = 'drive/My Drive/AI/Dataset/Open Images Dataset v4 (Bounding Boxes)/train'
#         fileName = row['path']
#         className = row['objects_name']
#         f.write(fileName + ',' + str(x1) + ',' + str(y1) + ',' + str(x2) + ',' + str(y2) + ',' + className + '\n')

In [ ]:
train_img_ids = train_df["path"].head(50).unique()

In [ ]:
# print(train_img_ids)

In [ ]:
for img_id in train_img_ids:
    
#     print(img_id+'.jpg')
#     img_id= img_id+'.jpg'
    plot_bbox(img_id)

Output hidden; open in https://colab.research.google.com to view.